从含有pose信息的文本文件里，找出指定文件夹中所有图像的pose信息，并记录。

In [124]:
import os

In [125]:
def read_file_list(filename):
    """
    Reads a trajectory from a text file. 
    
    File format:
    The file format is "stamp d1 d2 d3 ...", where stamp denotes the time stamp (to be matched)
    and "d1 d2 d3.." is arbitary data (e.g., a 3D position and 3D orientation) associated to this timestamp. 
    
    Input:
    filename -- File name
    
    Output:
    dict -- dictionary of (stamp,data) tuples
    
    """
    file = open(filename)
    data = file.read()
    lines = data.replace("\t"," ").split("\n") 
    list = [[v.strip() for v in line.split(" ") if v.strip()!=""] for line in lines if len(line)>0 and line[0]!="#"]
    #list = [[float(l[0]),float(l[1]),float(l[2]),float(l[3]),float(l[4]),float(l[5]),float(l[6]),float(l[7])] for l in list if len(l)>1]
    list = [(l[0],l[1:]) for l in list if len(l)>1]
    return dict(list)

In [126]:
def read_image_list(image_dir):
    '''
    Input:
    filename -- File name
    
    Output:
    list -- list of imagenames e.g.['png1','png2',...]
    
    '''
    for root, dirs, files in os.walk(image_dir):
        #print(root) 当前目录路径
        #print(dirs) 当前目录下所有子目录
        #print(files) #当前目录下所有非目录的子文件
        #files = [float(i) for i in files]
        files = [v.split(".png") for v in files]
        data=[i[0] for i in files]
        return data

In [127]:
def associate(filelist,poselist):
    '''
    Input:
    filelist -- list of imagenames
    poselist -- dict of pose information
    max_difference -- search radius for candidate generation
    
    Output:
    potential_matches -- list of matched tuples,e.g.[['1305032360.994081', '1305032360.9903'], ['1305032365.99406', '1305032365.9903'],...]
    '''
    poselist_keys=poselist.keys()
    potential_matches=[]
    
    for a in filelist:
        timestamp=1
        for b in poselist_keys:
            if abs(float(a)-float(b))<timestamp:
                timestamp=abs(float(a)-float(b))
                potential_match=[a,b]
        potential_matches.append(potential_match)
    return potential_matches

In [128]:
'''
list1 以字典形式存放groundtruth
image_dir 存放所有待匹配照片的文件名
matched 存放匹配关系
'''
list1=read_file_list("/home/cyj/github_project/Keras-CNN-Image-Retrieval/all_data/TUM-rgbd/rgbd_dataset_freiburg1_plant/groundtruth.txt")
#print(list1)
image_dir=read_image_list("/home/cyj/github_project/Keras-CNN-Image-Retrieval/all_data/TUM-rgbd/rgbd_dataset_freiburg1_plant/test")
#print(image_dir)
matched=associate(image_dir,list1)
#print(matched)
groundturth_file="/home/cyj/github_project/Keras-CNN-Image-Retrieval/all_data/TUM-rgbd/rgbd_dataset_freiburg1_plant/test_pose.txt"

#将每张照片对应的最近的参数设置为当前照片的
with open(groundturth_file,"w",encoding="utf-8") as f:
    for i in matched:
        image_pose=str(i[0])+" "+list1[i[1]][0]+" "+list1[i[1]][1]+" "+list1[i[1]][2]+" "+list1[i[1]][3]+" "+list1[i[1]][4]+" "+list1[i[1]][5]+" "+list1[i[1]][6]+"\n"
        f.write(image_pose)
#按文件出现的先后排列
with open(groundturth_file,"r",encoding="utf-8") as f1:
    lines = f1.readlines()
    #print(lines)
lines.sort()
with open(groundturth_file,"w",encoding="utf-8") as f_w:
    for line in lines:
        f_w.write(line)